In [15]:
#!pip install pysam
# !pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.6 MB/s eta 0:00:00m eta 0:00:010:00:01


In [16]:

from Bio import SeqIO

In [13]:
fasta.read()

'>sequence_1\nATGCTGCTGCTCGTCGTC\n>sequence_2\nTTTATATATAAGTGTAGTAGA\n>sequence_3\nGGGCGCGCGAGATAATATATTAT'

In [27]:
count = 0
for seq_record in SeqIO.parse("lncRNA.fa", "fasta"):
    count = count + 1
print(count)

3828


In [58]:

sequences = [(str([rec.id]) , str(rec.seq)) for rec in SeqIO.parse("test.fasta", "fasta")]

In [59]:
sequences

[("['snce_1']", 'ATGCTGCTGCTCGTCGTC'),
 ("['sequence_2']", 'TTTATATATAAGTGTAGTAGA'),
 ("['sequence_3']", 'GGGCGCGCGAGATAATATATTAT'),
 ("['3']",
  'CGCTACGTCTTACGCTGGAGCTCTCATGGATCGGTTCGGTAGGGCTCGATCACATCGCTAGCCAT')]

In [47]:
sequences = []
descr = None
# here is the path of multifalsta file
with open("test.fasta") as file:
    line = file.readline()[:-1]  # always trim newline
    while line:
        if line[0] == '>':
            if descr:  # any sequence found yet?
                sequences.append((descr, seq))
            descr = str(line[1:].split('>'))
            seq = ''  # start a new sequence
        else:
            seq += line
        line = file.readline()[:-1]
    sequences.append((descr, seq))

In [48]:
sequences

[("['snce_1']", 'ATGCTGCTGCTCGTCGTC'),
 ("['sequence_2']", 'TTTATATATAAGTGTAGTAGA'),
 ("['sequence_3']", 'GGGCGCGCGAGATAATATATTAT'),
 ("['3']",
  'CGCTACGTCTTACGCTGGAGCTCTCATGGATCGGTTCGGTAGGGCTCGATCACATCGCTAGCCAT')]

In [51]:
listOfOrf = list()
for index, value in enumerate(sequences):  # looping over the fragments extracted
    frames = [] # storing the six frame translation that it zould be extacted from the fragments 
    dna = value[1]  # extract the fragment 
    description = value[0] #extact the desciption even were not use it, just for learning purpose
    reverseCdna = [] # storing the reverse compliments
    # create the positive frames
    # split the frames into codons for better performance
    frames.append([dna[i:i + 3] for i in range(0, len(dna), 3)])
    frames.append([dna[i:i + 3] for i in range(1, len(dna), 3)])
    frames.append([dna[i:i + 3] for i in range(2, len(dna), 3)])
    # reverse compliment of the fragment
    reverse = {"A": "T", "C": "G", "T": "A", "G": "C"}
    for i in range(len(dna)):
        reverseCdna.append(reverse[dna[-i - 1]]) if dna[-i - 1] in reverse.keys() else reverseCdna.append(dna[-i - 1])  # if any contamination found we keep it for further more check
    reverseCdna = ''.join(reverseCdna) # joining 
    # create the negative frames
    frames.append([reverseCdna[i:i + 3] for i in range(0, len(reverseCdna), 3)])
    frames.append([reverseCdna[i:i + 3] for i in range(1, len(reverseCdna), 3)])
    frames.append([reverseCdna[i:i + 3] for i in range(2, len(reverseCdna), 3)])

IndexError: string index out of range

In [39]:
for i in range(0,len(frames),1): #looping all the frames
    start=0
    while start <len(frames[i]): #looping each frame for start and stop codons 
        if frames[i][start]=="ATG" or frames[i][start]=="TTG" or frames[i][start]=="CTG" or frames[i][start]=="GTG":
            for stop in range(start+1,len(frames[i]),1):
                         if frames[i][stop]=="TAA" or  frames[i][stop]=="TAG" or  frames[i][stop]=="TGA" :
                                listOfOrf.append(frames[i][start:stop]) # retrieve the orf 
                                start=stop+1 # avoiding multiple start codons
                                break
        start+=1

In [40]:
listOfOrf

[['ATG', 'GAT', 'CGG', 'TTC', 'GGT', 'AGG', 'GCT', 'CGA', 'TCA', 'CAT', 'CGC'],
 ['ATG',
  'GCT',
  'AGC',
  'GAT',
  'GTG',
  'ATC',
  'GAG',
  'CCC',
  'TAC',
  'CGA',
  'ACC',
  'GAT',
  'CCA'],
 ['ATG'],
 ['ATG', 'AGA', 'GCT', 'CCA', 'GCG']]